<a href="https://colab.research.google.com/github/aysealemdar/DataScience/blob/master/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Challenge Customer Analytics**

Random Forest

In [0]:
import csv
import pandas as pd
from sklearn.ensemble import RandomForestRegressor 
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier

##1. Load Data
- Verbindung zu Google Drive wird erstellt, so dass auf die abgelegte Datenset zugegriffen werden kann 
- Danach wird CSV eingelesen und ausgegeben

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [140]:
train_data = pd.read_csv('/content/drive/My Drive/Customer Analytics/Datensätze/clean_train_daily.csv', sep=";",
                 header=None,
                 names=['Datum', 'Jahr', 'Monat','TagMonat', 'Kalenderwoche', 'Tag', 'Tagesnummer',
                        'Lokaler Feiertag', 'FeiertagID', 'Temperatur', 'Windgeschwindigkeit', 
                        'Feuchtigkeit', 'Schneefall', 'Niederschlag', 'Sonnenminuten', 'Schulferien', 
                        'FerienID', 'Umsatz'])
train_data = train_data.drop(train_data.index[[0]])
train_data.head()

,Datum,Jahr,Monat,TagMonat,Kalenderwoche,Tag,Tagesnummer,Lokaler Feiertag,FeiertagID,Temperatur,Windgeschwindigkeit,Feuchtigkeit,Schneefall,Niederschlag,Sonnenminuten,Schulferien,FerienID,Umsatz
1,02.01.2017,2017,1,2,1,Mo,1,NaN,0,0.805833333,3.280416667,84.66666667,0,3,188,Christmas holidays,1,8275.69157
2,03.01.2017,2017,1,3,1,Di,2,NaN,0,-2.075416667,1.700833333,85.125,0,0,390,Christmas holidays,1,3915.02667
3,04.01.2017,2017,1,4,1,Mi,3,NaN,0,-0.00375,10.35125,87,1,1.9,0,Christmas holidays,1,2195.397
4,05.01.2017,2017,1,5,1,Do,4,NaN,0,-2.39125,6.202083333,89.58333333,1,1.4,275,Christmas holidays,1,2272.40105
5,06.01.2017,2017,1,6,1,Fr,5,NaN,0,-8,3.671666667,81.08333333,0,0,522,Christmas holidays,1,4914.95375


##2. DataFrame
- DataFrame erstellen damit Random Forest ausgeführt werden kann
- Alle Features, welche nit numerisch sind müssen entfernt werden. Random Forest kann damit nicht umgehen. 

In [141]:
df=pd.DataFrame(train_data)
df.head()
df.drop('Tag', axis=1, inplace=True)
df.drop('Schulferien', axis=1, inplace=True)
df.drop('Datum', axis=1, inplace=True)
df.drop('Lokaler Feiertag', axis=1, inplace=True)
df.head()


,Jahr,Monat,TagMonat,Kalenderwoche,Tagesnummer,FeiertagID,Temperatur,Windgeschwindigkeit,Feuchtigkeit,Schneefall,Niederschlag,Sonnenminuten,FerienID,Umsatz
1,2017,1,2,1,1,0,0.805833333,3.280416667,84.66666667,0,3,188,1,8275.69157
2,2017,1,3,1,2,0,-2.075416667,1.700833333,85.125,0,0,390,1,3915.02667
3,2017,1,4,1,3,0,-0.00375,10.35125,87,1,1.9,0,1,2195.397
4,2017,1,5,1,4,0,-2.39125,6.202083333,89.58333333,1,1.4,275,1,2272.40105
5,2017,1,6,1,5,0,-8,3.671666667,81.08333333,0,0,522,1,4914.95375


In [142]:
print(df.iat[104,4])
type(df.iat[104,4])

7


str

##3. Create test and train data
[Quelle vom Code](https://www.codementor.io/@agarrahul01/multiclass-classification-using-random-forest-on-scikit-learn-library-hkk4lwawu
)
- mit 0:13 geben wir an was die Werte sind und 13 ist der Index für den Label also Umsatz.
- mit x und y definieren wir die variablen und den Label

In [143]:
X = df.iloc[:, 0:13].values
y = df.iloc[:, 13].values

print('The y: ')
print(y)
print('The independent features set: ')
print(X[:14,:])
print('The dependent variable: ')
print(y[:14])

The y: 
['8275.69157' '3915.02667' '2195.397' ... '5796.62099' '9309.89851'
 '8164.10073']
The independent features set: 
[['2017' '1' '2' '1' '1' '0' '0.805833333' '3.280416667' '84.66666667'
  '0' '3' '188' '1']
 ['2017' '1' '3' '1' '2' '0' '-2.075416667' '1.700833333' '85.125' '0'
  '0' '390' '1']
 ['2017' '1' '4' '1' '3' '0' '-0.00375' '10.35125' '87' '1' '1.9' '0' '1']
 ['2017' '1' '5' '1' '4' '0' '-2.39125' '6.202083333' '89.58333333' '1'
  '1.4' '275' '1']
 ['2017' '1' '6' '1' '5' '0' '-8' '3.671666667' '81.08333333' '0' '0'
  '522' '1']
 ['2017' '1' '7' '1' '6' '0' '-4.9075' '1.989166667' '72.375' '0' '0'
  '55' '1']
 ['2017' '1' '8' '1' '7' '0' '-0.835416667' '4.62875' '93.75' '1' '4.8'
  '0' '0']
 ['2017' '1' '9' '2' '1' '0' '-1.991666667' '1.780416667' '90' '0' '0'
  '232' '0']
 ['2017' '1' '10' '2' '2' '0' '-0.285' '5.119583333' '92.66666667' '1'
  '7.5' '0' '0']
 ['2017' '1' '11' '2' '3' '0' '-0.452083333' '10.74041667' '88.75' '0'
  '1.6' '217' '0']
 ['2017' '1' '12' '2' 

- Unterteilung der Daten in Trainings- und Testdaten dabei nehmen wir 0.2 also 20% für den Test der Rest ist Training.

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(y_test)

##4. Build Random Forest Model
- n_estimators ist ein Parameter der Anzahl Decision Trees festlegt, die gebaut werden sollen. 
- random_state ist der Seed der festgelegt wird. wenn dies mit random_state=0 (oder einem anderen Wert) aufgerufen wird, dann erhalten wir jedes Mal den gleichen Ergebnisgenerator.
- danach wird Model ausgeführt

In [133]:
regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X_train, y_train)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=20, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

##5. Predict
- Prediction von Test Labels

In [0]:
y_pred = regressor.predict(X_test)
print(y_pred)
print(y_test)

##6. Metrics & Results
- Unterschiedliche Metricen werden verwendet um die Performance zu beurteilen

In [0]:
print('Classification Report')
print(classification_report(y_test,y_pred))
print('Accuracy')
print(accuracy_score(y_test, y_pred))
print('Confusion matrix')
print(confusion_matrix(y_test,y_pred))

In [134]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 3961.5933858715603
Mean Squared Error: 26685443.683781717
Root Mean Squared Error: 5165.795551875985


**Der Mean Absolute Error beträgt 3961**

##Try with Classifier

----------------------------------------------------------------------------------


Wir wiederholen alle Schritte und Erhöhen die Anzahl Decision Trees

In [145]:
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [0]:
y_pred = classifier.predict(X_test)
print(y_pred)
print(y_test)

In [147]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 3318.765823486239
Mean Squared Error: 19329339.443925913
Root Mean Squared Error: 4396.514465337958


#ZWEITER VERSUCH
________________________________________________________________________________

[Quelle vom Codes](https://www.youtube.com/watch?v=ok2s1vV9XW0)

##3. Create test and train data
- train_data in Test und Train teilen 80% train und 20% test
- X sind unabhängige Variablen die Umsatz beinhalten,deswegen löschen wir Umsatz Spalte und bei Y sage ich es Zielvariable ist 'Umsatz'

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Umsatz'],axis='columns'),df.Umsatz,test_size=0.2)

- Nachprüfen, wie viele Daten in Train und Test sind.

In [24]:
print("train: "+str(len(X_train)))
print("test: "+str(len(X_test)))

train: 869
test: 218


##4. Build Random Forest, fit and see the performance

[Quelle vom Code](https://www.geeksforgeeks.org/random-forest-regression-in-python/)
- Random Forest wird aufgebaut mit 100 Decision Trees

In [31]:
regressor_01 = RandomForestRegressor(n_estimators = 100, random_state = 0) 
regressor_01.fit(X_train,y_train) 

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

- erstes Ergebnis ist um die 78%

In [32]:
regressor_01.score(X_test,y_test)

0.784426732210069

- Gemäss Recherche heisst es mehr Decision Trees = Bessere Leistung. Dies probieren wir aus und erhöhen Decision Trees auf 200

In [148]:

regressor_02 = RandomForestRegressor(n_estimators = 200, random_state = 0) 
regressor_02.fit(X_train,y_train) 

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=200, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

- Im Score erhalten wir einen Wert von 0.78. 

In [37]:
regressor_02.score(X_test,y_test)

0.7823571204556975

In [0]:
y_pred = classifier.predict(X_test)
print(y_pred)
print(y_test)

In [150]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 3318.765823486239
Mean Squared Error: 19329339.443925913
Root Mean Squared Error: 4396.514465337958


- Das Ergebnis ist nicht besser. Wir versuchen aber nochmal den Parameter zu ändern. Dieses mal stellen wir es auf 50

In [39]:
regressor_03 = RandomForestRegressor(n_estimators = 50, random_state = 0) 
regressor_03.fit(X_train,y_train) 

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [45]:
regressor_03.score(X_test,y_test)

0.786901368277956

- Wir haben minim besseres Ergebnis

- Jetzt geben wir die predicted Werte mal aus

In [0]:
y_pred = regressor_03.predict(X_test)
y_pred

In [0]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

##MAKE PREDICTION WITH HOLDOUT DATASET WITHOUT LABELS

Vorgegebene Excelliste füllen

In [152]:
test_data = pd.read_csv('/content/drive/My Drive/Customer Analytics/Datensätze/clean_TEST_daily.csv', sep=";",
                 header=None,
                 names=['Datum', 'Jahr', 'Monat','TagMonat', 'Kalenderwoche', 'Tag', 'Tagesnummer',
                        'Lokaler Feiertag', 'FeiertagID', 'Temperatur', 'Windgeschwindigkeit', 
                        'Feuchtigkeit', 'Schneefall', 'Niederschlag', 'Sonnenminuten', 'Schulferien', 
                        'FerienID'])
test_data = test_data.drop(test_data.index[[0]])
test_data.head()

,Datum,Jahr,Monat,TagMonat,Kalenderwoche,Tag,Tagesnummer,Lokaler Feiertag,FeiertagID,Temperatur,Windgeschwindigkeit,Feuchtigkeit,Schneefall,Niederschlag,Sonnenminuten,Schulferien,FerienID
1,03.01.2020,2020,1,3,1,Fr,5,NaN,0,4.992083333,4.213333333,73.75,0,1.5,116,Christmas holidays,1
2,04.01.2020,2020,1,4,1,Sa,6,NaN,0,3.777916667,4.4925,86.54166667,0,2,179,Christmas holidays,1
3,05.01.2020,2020,1,5,1,So,7,NaN,0,0.85125,1.970416667,87.58333333,0,0,436,NaN,0
4,06.01.2020,2020,1,6,2,Mo,1,NaN,0,2.36875,2.05625,78.375,0,0,386,NaN,0
5,07.01.2020,2020,1,7,2,Di,2,NaN,0,3.644583333,3.129166667,82.91666667,0,0.2,52,NaN,0


In [153]:
df2=pd.DataFrame(test_data)
df2.head()

,Datum,Jahr,Monat,TagMonat,Kalenderwoche,Tag,Tagesnummer,Lokaler Feiertag,FeiertagID,Temperatur,Windgeschwindigkeit,Feuchtigkeit,Schneefall,Niederschlag,Sonnenminuten,Schulferien,FerienID
1,03.01.2020,2020,1,3,1,Fr,5,NaN,0,4.992083333,4.213333333,73.75,0,1.5,116,Christmas holidays,1
2,04.01.2020,2020,1,4,1,Sa,6,NaN,0,3.777916667,4.4925,86.54166667,0,2,179,Christmas holidays,1
3,05.01.2020,2020,1,5,1,So,7,NaN,0,0.85125,1.970416667,87.58333333,0,0,436,NaN,0
4,06.01.2020,2020,1,6,2,Mo,1,NaN,0,2.36875,2.05625,78.375,0,0,386,NaN,0
5,07.01.2020,2020,1,7,2,Di,2,NaN,0,3.644583333,3.129166667,82.91666667,0,0.2,52,NaN,0


In [154]:
df2.drop('Tag', axis=1, inplace=True)
df2.drop('Schulferien', axis=1, inplace=True)
df2.drop('Datum', axis=1, inplace=True)
df2.drop('Lokaler Feiertag', axis=1, inplace=True)
df2.head()


,Jahr,Monat,TagMonat,Kalenderwoche,Tagesnummer,FeiertagID,Temperatur,Windgeschwindigkeit,Feuchtigkeit,Schneefall,Niederschlag,Sonnenminuten,FerienID
1,2020,1,3,1,5,0,4.992083333,4.213333333,73.75,0,1.5,116,1
2,2020,1,4,1,6,0,3.777916667,4.4925,86.54166667,0,2,179,1
3,2020,1,5,1,7,0,0.85125,1.970416667,87.58333333,0,0,436,0
4,2020,1,6,2,1,0,2.36875,2.05625,78.375,0,0,386,0
5,2020,1,7,2,2,0,3.644583333,3.129166667,82.91666667,0,0.2,52,0


In [155]:
i=1
pd.DataFrame(df2, index=[i])

,Jahr,Monat,TagMonat,Kalenderwoche,Tagesnummer,FeiertagID,Temperatur,Windgeschwindigkeit,Feuchtigkeit,Schneefall,Niederschlag,Sonnenminuten,FerienID
1,2020,1,3,1,5,0,4.992083333,4.213333333,73.75,0,1.5,116,1


In [0]:
df2['Umsatz']=0

In [0]:
i=1
while i >len(test_data):
  prediction= regressor.predict(pd.DataFrame(df2, index=[i]))
  i=i+1
  df2['Umsatz']=prediction
  pd.DataFrame(df2, index=[i])

In [101]:
#Creating the dependent variable class
factor = pd.factorize(df['Umsatz'])
df.Umsatz = factor[0]
definitions = factor[1]
print(df.Umsatz.head())
print(definitions)

1    0
2    1
3    2
4    3
5    4
Name: Umsatz, dtype: int64
Index(['8275.69157', '3915.02667', '2195.397', '2272.40105', '4914.95375',
       '7549.33936', '4602.86814', '3127.52671', '1104.81482', '1994.81483',
       ...
       '7061.67283', '8217.69491', '5942.31027', '6181.85766', '9053.12047',
       '13942.69882', '9577.4858', '5796.62099', '9309.89851', '8164.10073'],
      dtype='object', length=1087)
